In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

import torch.nn.functional as F

from torchvision.transforms.functional import to_pil_image
from tqdm import tqdm
from PIL import Image
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Đường dẫn tới 2 thư mục chứa ảnh
photo1_folder = '/content/drive/MyDrive/Data/photo'
photo2_folder = '/content/drive/MyDrive/Data/photo2'

# Đọc list.csv của cả hai thư mục
df1 = pd.read_csv(os.path.join(photo1_folder, 'list.csv'))
df2 = pd.read_csv(os.path.join(photo2_folder, 'list.csv'))

# Thêm cột đường dẫn đầy đủ tới ảnh
df1['filename'] = df1['filename'].apply(lambda x: os.path.join(photo1_folder, x))
df2['filename'] = df2['filename'].apply(lambda x: os.path.join(photo2_folder, x))

In [ ]:
df = pd.concat([df1, df2], ignore_index=True)
print(df['label'].value_counts())
df.round()

label
1    5947
0     934
Name: count, dtype: int64


,filename,label
0,/content/drive/MyDrive/Data/photo/not_live/df0...,0
1,/content/drive/MyDrive/Data/photo/live/3bfc4b6...,1
2,/content/drive/MyDrive/Data/photo/live/0f15878...,1
3,/content/drive/MyDrive/Data/photo/live/0a33de2...,1
4,/content/drive/MyDrive/Data/photo/live/9acfb09...,1
...,...,...
6876,/content/drive/MyDrive/Data/photo2/not_live/72...,0
6877,/content/drive/MyDrive/Data/photo2/live/a0d3c8...,1
6878,/content/drive/MyDrive/Data/photo2/not_live/sp...,0
6879,/content/drive/MyDrive/Data/photo2/live/b748b6...,1


In [ ]:
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

print(f"Số lượng mẫu trong tập train: {len(train_df)}")
print(f"Số lượng mẫu trong tập validate: {len(val_df)}")
print(f"Số lượng mẫu trong tập test: {len(test_df)}")

Số lượng mẫu trong tập train: 4816
Số lượng mẫu trong tập validate: 1032
Số lượng mẫu trong tập test: 1033


In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.loc[idx, 'filename']
        label = self.dataframe.loc[idx, 'label']

        try:
            image = Image.open(img_path).convert('RGB')
        except:
            print(f"Lỗi khi đọc ảnh: {img_path}, trả về ảnh đen.")
            image = Image.new('RGB', (224, 224), (0, 0, 0))

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])


In [ ]:
# Tạo dataset
train_dataset = CustomImageDataset(train_df, transform=transform_train)
val_dataset = CustomImageDataset(val_df, transform=transform_test)
test_dataset = CustomImageDataset(test_df, transform=transform_test)

# Tạo dataloader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)


In [ ]:
images, labels = next(iter(train_loader))
print(images.shape)  # [batch_size, 3, 224, 224]
print(labels[:5])


torch.Size([32, 3, 224, 224])
tensor([1, 1, 1, 1, 0])


In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # [B, 3, 224, 224] -> [B, 32, 224, 224]
            nn.ReLU(),
            nn.MaxPool2d(2),  # -> [B, 32, 112, 112]

            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # -> [B, 64, 112, 112]
            nn.ReLU(),
            nn.MaxPool2d(2),  # -> [B, 64, 56, 56]

            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # -> [B, 128, 56, 56]
            nn.ReLU(),
            nn.MaxPool2d(2)   # -> [B, 128, 28, 28]
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),               # -> [B, 128*28*28]
            nn.Linear(128 * 28 * 28, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1)           # Đầu ra 1 neuron cho bài toán nhị phân
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x  # Không sigmoid vì dùng BCEWithLogitsLoss


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN().to(device)
criterion = nn.BCEWithLogitsLoss()  # Kết hợp sigmoid + binary cross entropy
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(loader):
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)  # [B] -> [B,1]

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(loader.dataset)
    return epoch_loss

def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)

            preds = torch.sigmoid(outputs) >= 0.5
            correct += (preds == labels.bool()).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / len(loader.dataset)
    accuracy = correct / total
    return epoch_loss, accuracy


In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)

    print(f"[Epoch {epoch+1}/{EPOCHS}] Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")


  0%|          | 0/151 [00:00<?, ?it/s]

Lỗi khi đọc ảnh: /content/drive/MyDrive/Data/photo/live/08cb5b00-b225-11ed-8e2a-b42e9920aa06_face_image_live_08c3a310-b225-11ed-b50d-b42e9920aa05.jpg, trả về ảnh đen.


 10%|▉         | 15/151 [01:37<11:57,  5.27s/it]

Lỗi khi đọc ảnh: /content/drive/MyDrive/Data/photo/live/7d040a34-b1cc-11ed-b531-b42e9920aa06_face_image_live_7cfe91bc-b1cc-11ed-80d1-b42e9920aa05.jpg, trả về ảnh đen.


 43%|████▎     | 65/151 [06:45<07:23,  5.15s/it]

...

Lỗi khi đọc ảnh: /content/drive/MyDrive/Data/photo2/live/a1c1435c-b1f2-11ed-95c0-b42e9920aa06_face_image_live_a1ba64d8-b1f2-11ed-80d1-b42e9920aa05.jpg, trả về ảnh đen.


 79%|███████▉  | 119/151 [01:23<00:23,  1.37it/s]

Lỗi khi đọc ảnh: /content/drive/MyDrive/Data/photo/live/08cb5b00-b225-11ed-8e2a-b42e9920aa06_face_image_live_08c3a310-b225-11ed-b50d-b42e9920aa05.jpg, trả về ảnh đen.


100%|██████████| 151/151 [01:46<00:00,  1.42it/s]


Lỗi khi đọc ảnh: /content/drive/MyDrive/Data/photo/live/5b861bb4-b2e8-11ed-95c0-b42e9920aa06_face_image_live_5b668e2a-b2e8-11ed-92d3-b42e9920aa05.jpg, trả về ảnh đen.
Lỗi khi đọc ảnh: /content/drive/MyDrive/Data/photo/live/8de73722-b221-11ed-95c0-b42e9920aa06_face_image_live_8ddf2f14-b221-11ed-92d3-b42e9920aa05.jpg, trả về ảnh đen.
Lỗi khi đọc ảnh: /content/drive/MyDrive/Data/photo2/live/b1edffac-b1d1-11ed-a57b-b42e9920aa06_face_image_live_b1e4897c-b1d1-11ed-8512-b42e9920aa05.jpg, trả về ảnh đen.
[Epoch 10/10] Train Loss: 0.0686 | Val Loss: 0.1708 | Val Acc: 0.9516


In [ ]:
test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")


Lỗi khi đọc ảnh: /content/drive/MyDrive/Data/photo/live/99c8b1e6-b31c-11ed-b17c-b42e9920aa06_face_image_live_99c5bec8-b31c-11ed-a25f-b42e9920aa05.jpg, trả về ảnh đen.
Test Loss: 0.1283 | Test Accuracy: 0.9545
